<a href="https://colab.research.google.com/github/ismab09/basededatos/blob/main/Base_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3
import os


# Ruta de la base de datos
db_path = 'base_de_datos_de_futbol.db'

# Crear o conectar a la base de datos
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Crear las tablas solo si no existen
cursor.execute('''
CREATE TABLE IF NOT EXISTS equipos (
    nombre TEXT PRIMARY KEY
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS jugadores (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    dorsal INTEGER NOT NULL,
    nombre TEXT NOT NULL,
    equipo_nombre TEXT,
    grl INTEGER,
    FOREIGN KEY (equipo_nombre) REFERENCES equipos (nombre)
)
''')

# Limpiar duplicados de jugadores
cursor.execute('''
DELETE FROM jugadores
WHERE id NOT IN (
    SELECT MIN(id)
    FROM jugadores
    GROUP BY dorsal, nombre, equipo_nombre, grl
)
''')

# Insertar equipos solo si no existen
equipos = ['Arsenal','Aston Villa','Bournemouth','Brentford','Brighton & Hove Albion','Chelsea','Crystal Palace',
           'Everton','Fulham','Ipswich Town','Leicester City','Liverpool','Manchester City','Manchester United',
           'Newcastle United','Nottingham Forest','Southampton','Tottenham Hotspur','West Ham United','Wolverhampton']
for equipo in equipos:
    cursor.execute('INSERT OR IGNORE INTO equipos (nombre) VALUES (?)', (equipo,))

# Insertar jugadores solo si no existen
jugadores = [
    (7, 'Bukayo Saka', 'Arsenal', 87),

]

for jugador in jugadores:
    # Verificar si el jugador ya existe antes de insertar
    cursor.execute('''
    SELECT 1 FROM jugadores WHERE dorsal = ? AND nombre = ? AND equipo_nombre = ? AND grl = ?
    ''', jugador)

    if cursor.fetchone() is None:
        cursor.execute('INSERT INTO jugadores (dorsal, nombre, equipo_nombre, grl) VALUES (?, ?, ?, ?)', jugador)

# Guardar los cambios
conn.commit()


# Consultar y mostrar todos los equipos en orden alfabético
cursor.execute('SELECT nombre FROM equipos ORDER BY nombre')
equipos = cursor.fetchall()

# Mostrar menú de opciones para elegir un equipo
print("Seleccione un equipo para mostrar sus jugadores:")
for i, equipo in enumerate(equipos, start=1):
    print(f"{i}. {equipo[0]}")

# Obtener la opción del usuario
opcion = int(input("\nIngrese el número del equipo que desea elegir: "))

# Validar la opción ingresada
if opcion < 1 or opcion > len(equipos):
    print("Opción no válida.")
else:
    # Obtener el nombre del equipo seleccionado
    equipo_nombre = equipos[opcion - 1][0]
    print(f"\nEquipo seleccionado: {equipo_nombre}")

    # Consultar jugadores del equipo seleccionado, ordenados por dorsal
    cursor.execute('SELECT dorsal, nombre, grl FROM jugadores WHERE equipo_nombre = ? ORDER BY dorsal', (equipo_nombre,))
    jugadores = cursor.fetchall()

    # Mostrar los jugadores del equipo seleccionado
    if jugadores:  # Verifica si hay jugadores para mostrar
        for jugador in jugadores:
            dorsal, nombre, grl = jugador
            print(f"  - {dorsal} {nombre} {grl}")
    else:
        print("  - No hay jugadores para mostrar.")

# Cerrar la conexión
conn.close()



Seleccione un equipo para mostrar sus jugadores:
1. Arsenal
2. Aston Villa
3. Bournemouth
4. Brentford
5. Brighton & Hove Albion
6. Chelsea
7. Crystal Palace
8. Everton
9. Fulham
10. Ipswich Town
11. Leicester City
12. Liverpool
13. Manchester City
14. Manchester United
15. Newcastle United
16. Nottingham Forest
17. Southampton
18. Tottenham Hotspur
19. West Ham United
20. Wolverhampton


import sqlite3
import os
import shutil

# Ruta de la base de datos
db_path = 'base_de_datos_de_futbol.db'
backup_db_path = 'base_de_datos_de_futbol_backup.db'

# Función para verificar la integridad de la base de datos
def check_database_integrity(db_path):
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        cursor.execute('PRAGMA integrity_check')
        result = cursor.fetchone()
        conn.close()
        return result[0] == 'ok'
    except sqlite3.DatabaseError as e:
        print(f"Error al verificar la base de datos: {e}")
        return False

# Intentar hacer una copia de seguridad y verificar la integridad
if not check_database_integrity(db_path):
    print("La base de datos está corrupta. Intentando recuperar desde una copia de seguridad.")

    # Hacer una copia de seguridad de la base de datos corrupta
    shutil.copy(db_path, backup_db_path)

    # Intentar verificar la base de datos copiada
    if not check_database_integrity(backup_db_path):
        print("La base de datos copiada también está corrupta. Eliminando la base de datos corrupta.")
        # Eliminar la base de datos corrupta
        if os.path.exists(db_path):
            os.remove(db_path)

    # Crear una nueva base de datos
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Crear las tablas con nombre como clave primaria
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS equipos (
        nombre TEXT PRIMARY KEY
    )
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS jugadores (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nombre TEXT NOT NULL,
        equipo_nombre TEXT,
        FOREIGN KEY (equipo_nombre) REFERENCES equipos (nombre)
    )
    ''')

    # Guardar los cambios y cerrar la conexión
    conn.commit()
    conn.close()

    print("Base de datos y tablas recreadas con éxito.")

# Conectar a la base de datos
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Insertar equipos
equipos = ['Chelsea', 'Liverpool', 'Manchester City']
for equipo in equipos:
    cursor.execute('INSERT INTO equipos (nombre) VALUES (?)', (equipo,))

# Guardar los cambios
conn.commit()

# Insertar jugadores
# Usar nombres de equipos en lugar de IDs
jugadores = [
    ('Robert Sánchez', 'Chelsea'),
    ('Cole Palmer', 'Chelsea'),
    ('Rodri', 'Manchester City')
]

for jugador in jugadores:
    cursor.execute('INSERT INTO jugadores (nombre, equipo_nombre) VALUES (?, ?)', jugador)

# Guardar los cambios
conn.commit()

# Actualizar el nombre de un jugador
cursor.execute('UPDATE jugadores SET nombre = ? WHERE id = ?', ('Nuevo Jugador', 1))

# Guardar los cambios
conn.commit()

# Eliminar un jugador
cursor.execute('DELETE FROM jugadores WHERE id = ?', (2,))

# Guardar los cambios
conn.commit()

# Consultar y mostrar jugadores por equipo
cursor.execute('SELECT * FROM equipos')
equipos = cursor.fetchall()

for equipo in equipos:
    equipo_nombre = equipo[0]
    print(f"\nEquipo: {equipo_nombre}")
    
    cursor.execute('SELECT * FROM jugadores WHERE equipo_nombre = ?', (equipo_nombre,))
    jugadores = cursor.fetchall()
    
    for jugador in jugadores:
        print(f"  - {jugador[1]}")

# Cerrar la conexión
conn.close()


# Cargar JSON

In [ ]:
import json
import sqlite3

# Conexión a la base de datos (crea la base de datos si no existe)
conn = sqlite3.connect('jugadores.db')
cursor = conn.cursor()

# Crear tabla si no existe (agregamos el campo avatarUrl)
cursor.execute('''
CREATE TABLE IF NOT EXISTS players (
    id INTEGER PRIMARY KEY,
    overallRating INTEGER,
    name TEXT,
    nationality_id INTEGER,
    nationality_label TEXT,
    nationality_imageUrl TEXT,
    team_id INTEGER,
    team_label TEXT,
    team_imageUrl TEXT,
    position_id TEXT,
    position_shortLabel TEXT,
    avatarUrl TEXT
)
''')

# Función para insertar un jugador en la base de datos
def insertar_jugador(jugador):
    # Preparar los datos
    player_id = jugador['id']
    overall_rating = jugador['overallRating']

    # Verificar si tiene commonName, si no, usar firstName y lastName
    name = jugador['commonName'] if jugador['commonName'] else f"{jugador['firstName']} {jugador['lastName']}"

    # Nacionalidad
    nationality_id = jugador['nationality']['id']
    nationality_label = jugador['nationality']['label']
    nationality_imageUrl = jugador['nationality']['imageUrl']

    # Equipo
    team_id = jugador['team']['id']
    team_label = jugador['team']['label']
    team_imageUrl = jugador['team']['imageUrl']

    # Posición
    position_id = jugador['position']['id']
    position_shortLabel = jugador['position']['shortLabel']

    # Avatar URL
    avatar_url = jugador.get('avatarUrl', None)  # Usamos .get() para evitar errores si el campo no existe

    # Insertar en la base de datos
    cursor.execute('''
    INSERT INTO players (id, overallRating, name, nationality_id, nationality_label, nationality_imageUrl,
                         team_id, team_label, team_imageUrl, position_id, position_shortLabel, avatarUrl)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (player_id, overall_rating, name, nationality_id, nationality_label, nationality_imageUrl,
          team_id, team_label, team_imageUrl, position_id, position_shortLabel, avatar_url))

    # Guardar los cambios
    conn.commit()

# Función para leer un archivo JSON con múltiples jugadores e insertarlos en la base de datos
def insertar_jugadores_desde_json(ruta_json):
    # Leer el archivo JSON
    with open(ruta_json, 'r', encoding='utf-8') as archivo:
        datos = json.load(archivo)

        # Para cada jugador en el JSON, lo insertamos en la base de datos
        for jugador in datos['items']:
            insertar_jugador(jugador)

# Ejemplo de uso: lectura de un archivo JSON e inserción de jugadores
ruta_json = 'equipos/0011_manchesterunited.json'  # Cambia esta ruta al nombre de tu archivo
insertar_jugadores_desde_json(ruta_json)

# Cerrar la conexión
conn.close()


# Consultar todos los jugadores

In [ ]:
import sqlite3
from IPython.display import display, HTML

# Conexión a la base de datos
conn = sqlite3.connect('jugadores.db')
cursor = conn.cursor()

# Función para mostrar texto con imagen alineada a la derecha del texto más largo
def mostrar_texto_con_imagen_a_derecha(texto, url_imagen, ancho_imagen="100px", alto_imagen="100px"):
    html_code = f"""
    <div style="display: flex; align-items: center;">
        <div style="min-width: 300px;"> <!-- Ajusta el min-width según sea necesario -->
            <p style="margin: 0;">{texto}</p>
        </div>
        <div style="margin-left: 10px;">
            <img src="{url_imagen}" style="width: {ancho_imagen}; height: {alto_imagen};">
        </div>
    </div>
    """
    display(HTML(html_code))

# Función para mostrar texto con imagen pequeña alineada correctamente (centrada)
def mostrar_texto_con_imagen_pequena(texto, url_imagen, ancho_imagen="20px", alto_imagen="20px"):
    html_code = f"""
    <div style="display: inline-flex; align-items: center;">
        <span style="margin-right: 5px; line-height: {alto_imagen};">{texto}</span>
        <img src="{url_imagen}" style="width: {ancho_imagen}; height: {alto_imagen}; vertical-align: middle;">
    </div>
    """
    display(HTML(html_code))

# Función para leer y mostrar los jugadores de la base de datos
def mostrar_jugadores():
    # Ejecutar una consulta SQL para seleccionar todos los jugadores
    cursor.execute('SELECT * FROM players')

    # Obtener todos los resultados
    jugadores = cursor.fetchall()

    # Verificar si hay jugadores en la base de datos
    if jugadores:
        for jugador in jugadores:
            texto_principal = f"""
            ID: {jugador[0]}<br>
            Overall Rating: {jugador[1]}<br>
            Nombre: {jugador[2]}<br>
            Nacionalidad ID: {jugador[3]},  |  Nacionalidad: {jugador[4]}<br>
            Equipo ID: {jugador[6]},  |  Equipo: {jugador[7]}<br>
            Posición ID: {jugador[9]}  |  Posición: {jugador[10]}
            """

            # Mostrar texto con la imagen del jugador a la derecha del texto más largo
            mostrar_texto_con_imagen_a_derecha(texto_principal, jugador[11])

            # Mostrar imagen de la nacionalidad centrada correctamente con el texto
            mostrar_texto_con_imagen_pequena("Nacionalidad:", jugador[5], ancho_imagen="20px", alto_imagen="20px")

            # Mostrar escudo del equipo con tamaño más pequeño y alineado correctamente
            mostrar_texto_con_imagen_pequena("Escudo del equipo:", jugador[8], ancho_imagen="30px", alto_imagen="30px")

            print("-" * 50)
    else:
        print("No hay jugadores en la base de datos.")

# Llamar a la función para mostrar los jugadores
mostrar_jugadores()

# Cerrar la conexión a la base de datos
conn.close()


# Consultar con filtros


In [ ]:
import sqlite3
from IPython.display import display, HTML

# Conexión a la base de datos
conn = sqlite3.connect('jugadores.db')
cursor = conn.cursor()

# Función para mostrar texto con imagen alineada a la derecha del texto más largo
def mostrar_texto_con_imagen_a_derecha(texto, url_imagen, ancho_imagen="100px", alto_imagen="100px"):
    html_code = f"""
    <div style="display: flex; align-items: center;">
        <div style="min-width: 300px;"> <!-- Ajusta el min-width según sea necesario -->
            <p style="margin: 0;">{texto}</p>
        </div>
        <div style="margin-left: 10px;">
            <img src="{url_imagen}" style="width: {ancho_imagen}; height: {alto_imagen};">
        </div>
    </div>
    """
    display(HTML(html_code))

# Función para mostrar texto con imagen pequeña alineada correctamente (centrada)
def mostrar_texto_con_imagen_pequena(texto, url_imagen, ancho_imagen="20px", alto_imagen="20px"):
    html_code = f"""
    <div style="display: inline-flex; align-items: center;">
        <span style="margin-right: 5px; line-height: {alto_imagen};">{texto}</span>
        <img src="{url_imagen}" style="width: {ancho_imagen}; height: {alto_imagen}; vertical-align: middle;">
    </div>
    """
    display(HTML(html_code))

# Función para mostrar los equipos disponibles en orden ascendente por ID, junto con la cantidad de jugadores
def mostrar_equipos_disponibles():
    # Ejecutar una consulta SQL para contar jugadores por equipo, ordenados por team_id en forma ascendente
    cursor.execute('''
    SELECT team_id, team_label, COUNT(*) as cantidad_jugadores
    FROM players
    GROUP BY team_id, team_label
    ORDER BY team_id ASC
    ''')

    # Obtener todos los equipos
    equipos = cursor.fetchall()

    # Mostrar los equipos en orden ascendente por ID, junto con la cantidad de jugadores
    print("Equipos disponibles:")
    for equipo in equipos:
        # Modificar el nombre del equipo "man utd" a "Manchester United"
        nombre_equipo = "Manchester United" if equipo[1].lower() == "man utd" else equipo[1]
        print(f"ID: {equipo[0]} | {nombre_equipo} | Jugadores: {equipo[2]}")

    # Solicitar al usuario que seleccione un equipo por ID
    equipo_id = input("\nIngrese el ID del equipo que desea ver: ")
    print('\n')

    return equipo_id

# Función para mostrar jugadores del equipo seleccionado
def mostrar_jugadores(equipo_id):
    # Ejecutar una consulta SQL para seleccionar jugadores por equipo
    cursor.execute('SELECT * FROM players WHERE team_id = ?', (equipo_id,))

    # Obtener todos los resultados
    jugadores = cursor.fetchall()

    # Verificar si hay jugadores en la base de datos para el equipo seleccionado
    if jugadores:
        for jugador in jugadores:
            # Modificar el nombre del equipo "man utd" a "Manchester United"
            nombre_equipo = "Manchester United" if jugador[7].lower() == "man utd" else jugador[7]
            texto_principal = f"""
            ID: {jugador[0]}<br>
            Overall Rating: {jugador[1]}<br>
            Nombre: {jugador[2]}<br>
            Nacionalidad ID: {jugador[3]}  |  Bandera: {jugador[4]}<br>
            Equipo ID: {jugador[6]}  |  Equipo: {nombre_equipo}<br>
            Posición ID: {jugador[9]}  |  Posición: {jugador[10]}
            """

            # Mostrar texto con la imagen del jugador a la derecha del texto más largo
            mostrar_texto_con_imagen_a_derecha(texto_principal, jugador[11])

            # Mostrar imagen de la nacionalidad centrada correctamente con el texto
            mostrar_texto_con_imagen_pequena("Nacionalidad:", jugador[5], ancho_imagen="20px", alto_imagen="20px")

            # Mostrar escudo del equipo con tamaño más pequeño y alineado correctamente
            mostrar_texto_con_imagen_pequena("Escudo del equipo:", jugador[8], ancho_imagen="30px", alto_imagen="30px")

            print("-" * 50)
    else:
        print("No hay jugadores en la base de datos para el equipo seleccionado.")

# Llamar a la función para mostrar los equipos disponibles y obtener el equipo seleccionado
equipo_seleccionado = mostrar_equipos_disponibles()

# Mostrar los jugadores del equipo seleccionado
mostrar_jugadores(equipo_seleccionado)

# Cerrar la conexión a la base de datos
conn.close()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
conn = sqlite3.connect('/content/drive/My Drive/jugadores.db')


# Prueba de funciones sin influir en la base de datos

In [ ]:
import sqlite3

# Conexión a la base de datos
conn = sqlite3.connect('jugadores.db')
cursor = conn.cursor()

# Función para calcular el total y promedio del overallRating de cada equipo
def calcular_grl_total_y_promedio():
    # Ejecutar una consulta SQL para calcular el total y promedio de overallRating por equipo
    cursor.execute('''
    SELECT team_id, team_label, SUM(overallRating) AS total_grl, AVG(overallRating) AS promedio_grl
    FROM players
    GROUP BY team_id, team_label
    ORDER BY team_id ASC
    ''')

    # Obtener todos los resultados
    resultados = cursor.fetchall()

    # Mostrar resultados
    print("Total y Promedio del Overall Rating por Equipo:")
    for resultado in resultados:
        # Modificar el nombre del equipo "man utd" a "Manchester United"
        nombre_equipo = "Manchester United" if resultado[1].lower() == "man utd" else resultado[1]
        print(f"ID: {resultado[0]} | Equipo: {nombre_equipo} | Total GRL: {resultado[2]} | Promedio GRL: {resultado[3]:.2f}")

# Llamar a la función para calcular y mostrar el total y promedio del overallRating
calcular_grl_total_y_promedio()

# Cerrar la conexión a la base de datos
conn.close()
